In [1]:
import os
import json
import re

In [2]:
def merge_details(backward_details):
    merged_list = []
    current_item = ""

    for i, item in enumerate(backward_details):
        if item.strip() and item.strip()[0].isdigit():
            j = 1
            while j < len(item.strip()) and item.strip()[j].isdigit():
                j += 1
            if j < len(item.strip()) and item.strip()[j] == "：":
                merged_list.append(current_item)
                current_item = item.strip()
            else:
                current_item += item.strip()
        else:
            current_item += item.strip()

    merged_list.append(current_item)
    merged_list.pop(0)
    return merged_list

In [3]:
def extract_tuples(string):
    tuples = []
    depth = 0
    start = None
    for i, char in enumerate(string):
        if char == "(" or char == "（" or char == "[":
            if depth == 0:
                start = i
            depth += 1
        elif char == ")" or char == "）" or char == "]":
            depth -= 1
            if depth == 0 and start is not None:
                tuples.append(string[start+1:i])
                start = None
    return tuples


def get_results(merged_list, input_list, entity_type_list, ent_extend_map, mode):
    input_list2ent = {}
    for idx, item in enumerate(input_list):
        input_list2ent[item] = entity_type_list[idx//10]

    ent_extend_index = 0
    cur_ent_extend_list = []
    A = []

    for idx, item in enumerate(merged_list):
        B = []
        cur_output_3 = item.split('Output:')
        cur_idx = cur_output_3[0].split('：')[0]
        cur_example = input_list[int(cur_idx)-1]
        cur_true_ent = input_list2ent[cur_example]
        if ent_extend_index == len(cur_ent_extend_list):
            ent_extend_index = 0

        cur_ent_extend_list = ent_extend_map[cur_true_ent]
        cur_extend_ent = cur_ent_extend_list[ent_extend_index]

        for output in cur_output_3[1:]:
            sep = ""
            if 'zh' in mode:
                output = output.replace("：", ":")
                if "生成实体列表:" in output:
                    sep = "生成实体列表:"
                elif "生成实体列表：" in output:
                    sep = "生成实体列表："
            elif 'en' in mode:
                if "Entity list:" in output:
                    sep = "Entity list:"
                elif "entity list:" in output:
                    sep = "entity list:"
                elif "Entity List:" in output:
                    sep = "Entity List:"
                elif "Generate an entity list:" in output:
                    sep = "Generate an entity list:"
                elif "Generated entity list:" in output:
                    sep = "Generated entity list:"
                elif "generate entity list:" in output:
                    sep = "generate entity list:"

            if sep:
                tmp = output.split(sep)[1]
                matches = extract_tuples(tmp)
                if matches:
                    for match in matches:
                        if 'zh' in mode:
                            triple = tuple(match.replace('，',',').strip().strip('（）').split(','))
                        elif 'en' in mode:
                            triple = tuple(match.strip().strip('()').split(','))
                        if len(triple) == 2:
                            B.append((int(cur_idx)-1, cur_extend_ent, triple[1].strip()))
                        else:
                            B.append((int(cur_idx)-1, cur_extend_ent, ''))
                else:
                    B.append((int(cur_idx)-1, cur_extend_ent, ''))
                
            else:
                B.append((int(cur_idx)-1, cur_extend_ent, ''))
                continue
        if B:
            A.append(B)    

        ent_extend_index += 1

    A = [list(set(item)) for item in A]
    
    with open(f'./results/backward_{mode}.txt','w',encoding='utf-8') as f:
        for item in A:
            f.write(str(item)+'\n')


def get_gold_results(merged_list, input_list, entity_type_list, mode):
    input_list2ent = {}
    for idx, item in enumerate(input_list):
        input_list2ent[item] = entity_type_list[idx//10]

    A = []

    for idx, item in enumerate(merged_list):
        B = []
        cur_output_3 = item.split('Output:')
        cur_idx = cur_output_3[0].split('：')[0]
        cur_example = input_list[int(cur_idx)-1]
        cur_true_ent = input_list2ent[cur_example]


        for output in cur_output_3[1:]:
            sep = ""
            if 'zh' in mode:
                output = output.replace("：", ":")
                if "生成实体列表:" in output:
                    sep = "生成实体列表:"
                elif "生成实体列表：" in output:
                    sep = "生成实体列表："
            elif 'en' in mode:
                if "Entity list:" in output:
                    sep = "Entity list:"
                elif "entity list:" in output:
                    sep = "entity list:"
                elif "Entity List:" in output:
                    sep = "Entity List:"
                elif "Generate an entity list:" in output:
                    sep = "Generate an entity list:"
                elif "Generated entity list:" in output:
                    sep = "Generated entity list:"
                elif "generate entity list:" in output:
                    sep = "generate entity list:"

            if sep:
                tmp = output.split(sep)[1]
                matches = extract_tuples(tmp)
                if matches:
                    for match in matches:
                        if 'zh' in mode:
                            triple = tuple(match.replace('，',',').strip().strip('（）').split(','))
                        elif 'en' in mode:
                            triple = tuple(match.strip().strip('()').split(','))
                        if len(triple) == 2:
                            B.append((int(cur_idx)-1, cur_true_ent, triple[1].strip()))
                        else:
                            B.append((int(cur_idx)-1, cur_true_ent, ''))
                else:
                    B.append((int(cur_idx)-1, cur_true_ent, ''))
                
            else:
                B.append((int(cur_idx)-1, cur_true_ent, ''))
                continue
        if B:
            A.append(B)    

    A = [list(set(item)) for item in A]
    
    with open(f'./results/backward_gold_{mode}.txt','w',encoding='utf-8') as f:
        for item in A:
            f.write(str(item)+'\n')

CMeEE

In [6]:
with open('./results/details_zh.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_chatgpt_zh.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/cmeee/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/cmeee/final_entity_extend_map_zh.json','r',encoding='utf-8') as f:
    ent_extend_map = json.load(f)

entity_type_list = ['dru', 'bod', 'pro', 'sym', 'equ', 'ite', 'dep', 'mic', 'dis']

# merged_list = merge_details(backward_details)
# get_results(merged_list, input_list, entity_type_list, ent_extend_map, 'zh')
merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, entity_type_list, 'chatgpt_zh')


ACE05

In [7]:
with open('./results/details_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_chatgpt_en.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/ace05/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/ace05/final_entity_extend_map_en.json','r',encoding='utf-8') as f:
    ent_extend_map = json.load(f)

with open('../ACE05/labels.json','r',encoding='utf-8') as f:
    entity_type_list = json.load(f)

# merged_list = merge_details(backward_details)
# get_results(merged_list, input_list, entity_type_list, ent_extend_map, 'en')

merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, entity_type_list, 'chatgpt_en')

Alpaca-CMeEE

In [5]:
# 处理反向验证的输出文件
with open('./results/details_alpaca_33B_zh.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_alpaca_33B_zh.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/cmeee/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/cmeee/final_entity_extend_map_alpaca_33B_zh.json','r',encoding='utf-8') as f:
    ent_extend_map = json.load(f)

entity_type_list = ['dru', 'bod', 'pro', 'sym', 'equ', 'ite', 'dep', 'mic', 'dis']

# merged_list = merge_details(backward_details)
# get_results(merged_list, input_list, entity_type_list, ent_extend_map, 'alpaca_33B_zh')
merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, entity_type_list, 'alpaca_33B_zh')


Alpaca-ACE05

In [6]:
with open('./results/details_alpaca_33B_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_alpaca_33B_en.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/ace05/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/ace05/final_entity_extend_map_alpaca_33B_en.json','r',encoding='utf-8') as f:
    ent_extend_map = json.load(f)

with open('../ACE05/labels.json','r',encoding='utf-8') as f:
    entity_type_list = json.load(f)

# merged_list = merge_details(backward_details)
# get_results(merged_list, input_list, entity_type_list, ent_extend_map, 'alpaca_33B_en')
# sum([len(ent_extend_map[key]) for key in entity_type_list])

merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, entity_type_list, 'alpaca_33B_en')

Llama2-ACE05

In [7]:
with open('./results/details_llama2_70B_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_llama2_70B_en.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/ace05/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/ace05/final_entity_extend_map_llama2_70B_en.json','r',encoding='utf-8') as f:
    ent_extend_map = json.load(f)

with open('../ACE05/labels.json','r',encoding='utf-8') as f:
    entity_type_list = json.load(f)

# merged_list = merge_details(backward_details)
# get_results(merged_list, input_list, entity_type_list, ent_extend_map, 'llama2_70B_en')
# sum([len(ent_extend_map[key]) for key in entity_type_list])

merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, entity_type_list, 'llama2_70B_en')

ChatGLM-CMeEE

In [8]:
# 处理反向验证的输出文件
with open('./results/details_chatglm_6B_zh.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_chatglm_6B_zh.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/cmeee/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/cmeee/final_entity_extend_map_chatglm_6B_zh.json','r',encoding='utf-8') as f:
    ent_extend_map = json.load(f)

entity_type_list = ['dru', 'bod', 'pro', 'sym', 'equ', 'ite', 'dep', 'mic', 'dis']

# merged_list = merge_details(backward_details)
# get_results(merged_list, input_list, entity_type_list, ent_extend_map, 'chatglm_6B_zh')
merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, entity_type_list, 'chatglm_6B_zh')

ChatGLM-ACE05

In [9]:
with open('./results/details_chatglm_6B_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_chatglm_6B_en.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/ace05/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/ace05/final_entity_extend_map_chatglm_6B_en.json','r',encoding='utf-8') as f:
    ent_extend_map = json.load(f)

with open('../ACE05/labels.json','r',encoding='utf-8') as f:
    entity_type_list = json.load(f)

# merged_list = merge_details(backward_details)
# get_results(merged_list, input_list, entity_type_list, ent_extend_map, 'chatglm_6B_en')
# sum([len(ent_extend_map[key]) for key in entity_type_list])
merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, entity_type_list, 'chatglm_6B_en')


GPT4-CMeEE

In [5]:
# 处理反向验证的输出文件
with open('./results/details_gpt4_zh.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./data/cmeee/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/cmeee/final_entity_extend_map_gpt4_zh.json','r',encoding='utf-8') as f:
    ent_extend_map = json.load(f)

entity_type_list = ['dru', 'bod', 'pro', 'sym', 'equ', 'ite', 'dep', 'mic', 'dis']

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, entity_type_list, ent_extend_map, 'gpt4_zh')

GPT4-ACE05

In [6]:
with open('./results/details_gpt4_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./data/ace05/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/ace05/final_entity_extend_map_gpt4_en.json','r',encoding='utf-8') as f:
    ent_extend_map = json.load(f)

with open('../ACE05/labels.json','r',encoding='utf-8') as f:
    entity_type_list = json.load(f)

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, entity_type_list, ent_extend_map, 'gpt4_en')


Baichuan-CMeEE

In [4]:
# 处理反向验证的输出文件
with open('./results/details_baichuan2_13B_zh.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_baichuan2_13B_zh.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/cmeee/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/cmeee/final_entity_extend_map_baichuan2_13B_zh.json','r',encoding='utf-8') as f:
    ent_extend_map = json.load(f)

entity_type_list = ['dru', 'bod', 'pro', 'sym', 'equ', 'ite', 'dep', 'mic', 'dis']

# merged_list = merge_details(backward_details)
# get_results(merged_list, input_list, entity_type_list, ent_extend_map, 'baichuan2_13B_zh')
merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, entity_type_list, 'baichuan2_13B_zh')

Baichuan-ACE05

In [7]:
with open('./results/details_baichuan2_13B_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_baichuan2_13B_en.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/ace05/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/ace05/final_entity_extend_map_baichuan2_13B_en.json','r',encoding='utf-8') as f:
    ent_extend_map = json.load(f)

with open('../ACE05/labels.json','r',encoding='utf-8') as f:
    entity_type_list = json.load(f)

# merged_list = merge_details(backward_details)
# get_results(merged_list, input_list, entity_type_list, ent_extend_map, 'baichuan2_13B_en')
# sum([len(ent_extend_map[key]) for key in entity_type_list])
merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, entity_type_list, 'baichuan2_13B_en')
